# Customer Retention Classifier

In [1]:
import math
import re
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Imputer
from scipy.stats import mode
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
TRAIN_DATA = pd.read_csv("../../../kaggle_data/customer_retention/midterm_train.csv")
TEST_DATA = pd.read_csv("../../../kaggle_data/customer_retention/midterm_test.csv")
X = TRAIN_DATA
X_combined = pd.concat([X, TEST_DATA])

### Clean Month Data

In [4]:
def cleanMonth(x):
    if isinstance(x, str):
        if x == 'Jun': return 'june'
        elif x == 'July': return 'july'
        elif x == 'Aug': return 'august'
        elif x == 'May': return 'may'
        elif x == 'Mar': return 'march'
        elif x == 'Apr': return 'april'
        elif x == 'sept.': return 'september'
        elif x == 'Feb': return 'february'
        elif x == 'Oct': return 'october'
        elif x == 'Nov': return 'november'
        elif x == 'January': return 'january'
        elif x == 'Dev': return 'december'

month = pd.Series([cleanMonth(x) for x in X_combined.x19], name='month')

### Clean Weekday Data

In [5]:
def cleanWeekday(x):
    if isinstance(x, str):
        if x == 'monday': return 'monday'
        elif x == 'tuesday': return 'tuesday'
        elif x == 'wednesday': return 'wednesday'
        elif x == 'thurday': return 'thursday'
        elif x == 'friday': return 'friday'

weekday = pd.Series([cleanWeekday(x) for x in X_combined.x43], name='weekday')

### Clean Region Data

In [6]:
def cleanRegion(x):
    if isinstance(x, str):
        if x == 'asia': return 'asia'
        elif x == 'euorpe': return 'europe'
        elif x == 'america': return 'america'

region = pd.Series([cleanRegion(x) for x in X_combined.x16], name='region')

### Parse Money to Float

In [7]:
def parseMoneyToFloat(x):
    if isinstance(x, str): return float(re.sub('[$,()]', '', x))

X_combined.x44 = [parseMoneyToFloat(x) for x in X_combined.x44]

### Parse Percent to Float

In [8]:
def parsePercentToFloat(x):
    if isinstance(x, str): return float(re.sub('%', '', x)) / 100.00

X_combined.x09 = [parsePercentToFloat(x) for x in X_combined.x09]

### Imputation  Strategies

1. **df0**: Numerical: Drop, Categorical: Drop
2. **df1**: Numerical: Mean, Categorical: Frequency
3. **df2**: Numerical: Median, Categorical: Frequency
4. **df3**: k-Nearest Neighbors
5. **df4**: Random Forest

In [9]:
X_numeric = X_combined.copy()
X_numeric.drop(['x16', 'x19', 'x43'], axis=1, inplace=True)

In [10]:
df1 = X_numeric.copy()

### Impute Missing Categorical Data with Most Frequent Value & Get Dummies

In [11]:
month.value_counts()[[0]]

june    55795
Name: month, dtype: int64

In [12]:
weekday.value_counts()[[0]]

wednesday    126413
Name: weekday, dtype: int64

In [13]:
region.value_counts()[[0]]

asia    173129
Name: region, dtype: int64

In [14]:
month.fillna('june', inplace=True)
weekday.fillna('wednesday', inplace=True)
region.fillna('asia', inplace=True)

### df1: Numerical: Mean, Categorical: Most Frequent

In [15]:
y_df1 = df1.pop('y')

In [16]:
meanImputer = Imputer(strategy='mean', copy=False)
meanImputer.fit_transform(df1)

array([[ -5.90070664e+00,   4.69388657e-01,   4.73306785e+00, ...,
          4.59280348e+00,  -3.44776760e-02,   3.68040134e+00],
       [ -3.53039065e+00,   4.22981797e+00,  -4.61943288e+00, ...,
         -5.46246671e+00,  -2.29517509e-01,  -2.33294711e+00],
       [  1.00338855e+00,  -1.25474570e-02,   6.25050323e+00, ...,
         -1.14070719e+01,   6.39555117e+00,   3.54533178e+00],
       ..., 
       [  1.71043472e+01,   1.08815589e+01,   9.25710482e+00, ...,
          6.07498027e+00,  -2.24617902e-01,   5.16245808e+00],
       [  1.03181728e+01,  -8.79384077e+00,  -2.67099784e+00, ...,
         -7.78997798e+00,   1.60228442e-01,  -2.71319396e+00],
       [ -3.75580866e+00,   6.22956464e+00,  -1.28682043e+00, ...,
          6.10818001e+01,   1.34704559e+00,  -7.67326474e-01]])

In [17]:
df1['y'] = y_df1
df1['month'] = month.copy()
df1['weekday'] = weekday.copy()
df1['region'] = region.copy()
df1 = pd.get_dummies(df1)

### Create Test/Train Splits

In [18]:
# df1: Numerical: Mean, Categorical: Most Frequent

# X TEST
X_test_df1 = df1[np.isnan(df1.y)].copy()
X_test_df1.drop('y', axis=1, inplace=True)

# Xy TRAIN
train_df1 = df1.dropna(subset=['y'])

# y TRAIN
y_train_df1 = train_df1['y'].copy()

# X TRAIN
X_train_df1 = train_df1.drop('y', axis=1)

### Random Forest Classifer with K-Fold Cross Validation and Grid Search

In [19]:
# set hyperparameters
n_estimators=[3, 5, 10, 50, 100]
criterion=['gini', 'entropy']
max_features=['auto', 'log2', None]
#min_samples_split=[2, 10, 100]
#min_samples_leaf=[1, 10, 100]
#min_weight_fraction_leaf=[0.0, 0.1, 0.2, 0.5]
#min_impurity_split=[1e-07, 1e-06, 1e-08]
#bootstrap=[True, False]
#oob_score=[False, True]

hyperparameters = dict(n_estimators=n_estimators, criterion=criterion, max_features=max_features)

In [20]:
# get random forest classifier
rfc = RandomForestClassifier(random_state=42, verbose=10, n_jobs=2)

In [21]:
# fit models to training sets

#df1
rfc_estimator_df1 = GridSearchCV(rfc, hyperparameters, scoring='roc_auc', cv=10, verbose=10, n_jobs=2)
rfc_estimator_df1.fit(X_train_df1, y_train_df1)
rfc_best_estimator_df1 = rfc_estimator_df1.best_estimator_
print(rfc_best_estimator_df1)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   10.7s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:   18.4s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:   25.5s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:   39.1s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:   59.9s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  1.5min
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  4.4min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  9.5min
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed: 12.5min
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed: 12.8min
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed: 14.0min
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed: 18.4min
[Parallel(n_jobs=2)]: Done 109 tasks      | elapsed: 23.4min
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed: 28.3min
[Parallel(n_jobs=2)]: Done 141 tasks      | elapsed: 66.9min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 106.8min
[Parallel(n_jobs=2)]: D

building tree 1 of 100building tree 2 of 100



[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   16.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100


...

[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed: 11.7min


building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100


[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed: 13.5min


building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=2, oob_score=False, random_state=42,
            verbose=10, warm_start=False)


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 14.4min finished


In [22]:
def getMeanAndConfidence(scores, scoring, name):
    mean_score = scores.mean()
    std_dev = scores.std()
    std_error = scores.std() / math.sqrt(scores.shape[0])
    ci =  2.262 * std_error
    lower_bound = mean_score - ci
    upper_bound = mean_score + ci
    print ("%s is %f +/-  %f" % (name, mean_score, ci))
    print ('95 percent probability that if this experiment were repeated over and over the average %s score would be between %f and %f' % (scoring, lower_bound, upper_bound))

In [23]:
# cross validation

#df1
roc_auc_df1 = cross_val_score(rfc_best_estimator_df1, X_train_df1, y_train_df1, cv=10, scoring='roc_auc')
getMeanAndConfidence(roc_auc_df1, 'roc_auc', 'df1')

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100


[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   13.9s


building tree 5 of 100
building tree 6 of 100


[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:   27.9s


building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100


...

[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed: 10.7min


building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed: 11.4min finished
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    0.1s


df1 is 0.967112 +/-  0.001103
95 percent probability that if this experiment were repeated over and over the average roc_auc score would be between 0.966009 and 0.968214


[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.1s finished


array([  6.69811141e-02,   9.52736815e-02,   7.29613243e-03,
         2.51687560e-02,   7.09343707e-03,   7.37361997e-02,
         7.29402244e-03,   7.26294587e-03,   2.32530853e-02,
         2.10776691e-03,   7.29850198e-03,   7.38578261e-03,
         7.26611532e-03,   5.34957480e-02,   6.57469962e-02,
         7.44437520e-03,   7.47453119e-03,   7.40410830e-03,
         7.25982408e-03,   7.38435716e-03,   7.21484931e-03,
         7.42289744e-03,   3.50330620e-02,   7.32704114e-03,
         3.38570118e-02,   7.33370231e-03,   7.11352262e-03,
         7.35972099e-03,   7.31924840e-03,   2.37763982e-02,
         7.30539652e-03,   5.75668140e-02,   6.64079568e-02,
         7.27351924e-03,   7.19421866e-03,   3.40198962e-02,
         7.32932885e-03,   7.30628642e-03,   7.34789141e-03,
         7.39428403e-03,   7.31677902e-03,   7.28006010e-03,
         2.99250564e-02,   2.77785386e-02,   3.26096805e-02,
         7.37405080e-03,   7.39611176e-03,   3.59241142e-04,
         4.71908857e-04,

In [28]:
X_test_df1.head()

,x00,x01,x02,x03,x04,x05,x06,x07,x08,x09,x10,x11,x12,x13,x14,x15,x17,x18,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x44,x45,x46,x47,x48,x49,month_april,month_august,month_december,month_february,month_january,month_july,month_june,month_march,month_may,month_november,month_october,month_september,weekday_friday,weekday_monday,weekday_thursday,weekday_tuesday,weekday_wednesday,region_america,region_asia,region_europe
0,13.864410,0.986217,6.835099,12.129012,1.141835,0.257978,6.119007,-0.217401,-0.913796,0.0000,-0.419751,10.172915,-6.886273,-9.130198,-4.578509,7.960524,0.111165,-9.699845,-3.387555,1.911904,0.998081,-0.857490,-7.612868,-4.018814,7.209250,3.013906,-1.446528,-8.230251,-2.442596,-1.144471,-4.253401,-5.203326,2.729897,-1.202386,-2.511575,-13.927587,-14.171462,-2.149515,-11.997662,-3.078159,4.129892,376.12,21.431275,10.409146,10.712008,4.278259,-6.113483,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0
1,17.075344,-8.614119,9.730068,3.645163,18.618751,-1.134351,1.324885,2.750353,-21.323635,-0.0001,1.040758,-8.328905,21.474338,-82.372525,-19.870239,1.919702,0.513677,0.667307,2.426892,4.125267,-3.196495,1.820393,-30.719266,-0.332577,2.166615,-4.156138,-2.056772,1.107974,-7.752646,-26.706473,0.255282,20.459172,-19.764989,-0.103611,4.232745,-49.751647,8.288716,12.060118,0.782677,4.364007,-2.506411,1058.58,31.912299,23.853665,24.547705,1.835443,-0.698576,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0
2,-16.080927,1.490433,7.290475,12.770805,-2.401559,-0.143236,-9.110707,-0.044362,-22.508253,0.0000,-1.905429,17.536177,0.991342,-21.828601,-16.018098,-3.022159,0.229830,4.647811,-5.286930,-3.019935,-0.286837,1.681518,-31.103796,-8.774049,7.590720,-4.232454,8.458921,-7.666138,6.055294,-28.190131,15.473656,15.915722,-9.026852,1.171972,-5.391258,-59.082917,4.775361,1.834801,1.447475,2.394701,-4.988773,388.08,24.751632,9.235573,9.504289,-4.228346,-0.363838,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,8.955804,-2.612907,2.740679,9.704034,-1.524375,-4.001874,3.843455,0.358459,0.900835,-0.0001,-3.305960,11.099784,0.270078,-40.851392,-14.029412,-5.124856,-0.170633,-0.373501,2.284408,-2.855275,11.325377,3.110082,-19.147857,-8.226306,5.767890,6.093670,-7.830001,-6.567701,-3.929450,1.128238,3.463817,14.992986,-9.377136,-0.012398,4.087052,-61.945695,5.142323,12.855278,3.065369,-3.997100,6.548292,272.45,19.673348,-20.449157,-21.044141,-11.225027,9.176861,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,-1.243500,9.752493,-1.931690,1.224566,-18.678339,0.390264,10.615600,3.774666,-10.091425,-0.0001,1.023597,6.415119,-7.197756,-5.616595,10.993535,-2.497816,-0.196972,-12.111033,-0.197022,-1.789174,3.503165,5.784800,21.849473,-12.377823,0.727858,-5.169318,8.545741,1.213110,-9.659291,-12.638857,8.647821,10.183689,4.579249,0.586459,1.920605,13.387445,8.369600,-9.150596,-0.233386,4.280341,-3.315470,127.28,-5.184808,4.179362,4.300964,-1.475613,1.628260,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1


In [29]:
y = rfc_best_estimator_df1.predict(X_test_df1)

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  21 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  57 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done  81 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.4s finished


In [31]:
y.shape

(40000,)

In [32]:
y_pred = pd.Series(y)

In [34]:
y_pred.shape

(40000,)

In [35]:
y_pred.to_csv(path="../../../kaggle_data/customer_retention/y_predictions.csv")